In [1]:
import pandas as pd
import numpy as np
import locale
# plots
import matplotlib.pyplot as plt
from matplotlib import dates
# models
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sys
sys.path.append('/code/src/')
from my_functions import train_test_time_series, evaluate_forecast, load_dataframe_from_csv

### 1. Leer datos

In [2]:
path_load = "../../data/"
file_name = "processed/df_time_monthly.csv"
full_path = path_load + file_name


### 2 sarimax

Cómo construir automáticamente el modelo SARIMAX en python
Now let’s practice adding in an exogenous variable.

In [ ]:
df = load_dataframe_from_csv(full_path)
exog_vars = ['holidays_cont', 'month', 'year', 'days_in_month', 'is_first_month', 'is_last_month']
endog_var = 'cost'

exog = df[exog_vars]
endog = df[endog_var]
# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(df) * 0.8)  # Porcentaje de datos para entrenamiento
train_endog = endog[:train_size]
train_exog = exog[:train_size]
test_endog = endog[train_size:]
test_exog = exog[train_size:]

# Entrenar el modelo con los datos de entrenamiento
model = pm.auto_arima(train_endog, exogenous=train_exog, seasonal=True, trace=True)
# Obtener los valores del modelo seleccionado
order = model.order

print(order)

# Realizar la predicción en los datos de prueba
forecast = model.predict(n_periods=len(test_endog), exogenous=test_exog)

# Evaluar el rendimiento del modelo en los datos de prueba
# Aquí puedes utilizar las métricas adecuadas según el problema, como el error cuadrático medio (MSE) o el error absoluto medio (MAE).
forecast

Error Cuadrático Medio (MSE): El MSE mide el promedio de los errores al cuadrado entre las predicciones y los valores reales. Cuanto más bajo sea el MSE, mejor será el modelo en términos de ajuste a los datos observados.

Error Absoluto Medio (MAE): El MAE mide el promedio de los errores absolutos entre las predicciones y los valores reales. Esta métrica proporciona una medida del error promedio en la misma escala que los datos originales.

R-squared (R2): El coeficiente de determinación R-cuadrado representa la proporción de la varianza en la variable objetivo que es explicada por el modelo. Un valor más cercano a 1 indica un mejor ajuste del modelo a los datos observados.

Error Porcentual Absoluto Medio (MAPE): El MAPE mide el promedio de los errores porcentuales absolutos entre las predicciones y los valores reales. Esta métrica puede proporcionar una medida relativa del error en relación con el tamaño de los valores reales.

Es importante considerar que ninguna métrica de error es perfecta por sí misma. Por lo tanto, puede ser útil evaluar y comparar varias métricas en conjunto para tener una imagen más completa del rendimiento del modelo.

En tu pipeline, puedes aplicar diferentes modelos de aprendizaje automático, como regresión lineal, regresión de árboles de decisión, regresión de bosques aleatorios, regresión de Gradient Boosting, entre otros. Luego, puedes calcular y comparar las métricas de error mencionadas anteriormente para cada modelo y seleccionar aquel que presente el mejor rendimiento en función de tus objetivos y necesidades específicas.

Recuerda que también es importante considerar otros factores, como la interpretabilidad del modelo, la simplicidad, el tiempo de entrenamiento y la capacidad de escalabilidad, además de las métricas de error, al seleccionar el mejor modelo para tus necesidades particulares.

In [ ]:
df.head()

In [ ]:
evaluate_forecast(test_endog, forecast)

In [6]:
# Cargar y preparar los datos
data_sarima = load_dataframe_from_csv(full_path)
data_sarima.index.freq = 'M'
X = data_sarima.drop('cost', axis=1)  # Variables exógenas
y = data_sarima['cost']  # Variable objetivo
# Dividir los datos en conjuntos de entrenamiento y prueba basado en el índice de tiempo
train_size = int(len(data_sarima) * 0.8)
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

# Ajustar modelo Auto ARIMA
model_arima = pm.auto_arima(train_y, exogenous=train_X, seasonal=True)
y_pred_arima = model_arima.predict(n_periods=len(test_y), exogenous=test_X)
# Ajustar modelo SARIMA
order = model_arima.order
seasonal_order = model_arima.seasonal_order
print('order', order)
print('seasonal_order', seasonal_order)
# order=(1,1,0), seasonal_order=(0,1,0,12)
model_sarima = SARIMAX(train_y, exog=train_X, order=(1,1,0), seasonal_order=(0,1,0,12))
model_sarima_fit = model_sarima.fit()
# Obtener el resumen del modelo
print(model_sarima_fit.summary())
print("Model: SARIMA")
# Predecir con SARIMA
y_pred_sarima = model_sarima_fit.predict(start=len(train_y), end=len(train_y) + len(test_y) - 1, exog=test_X)
evaluate_forecast(test_y, y_pred_sarima)

order (1, 1, 0)
seasonal_order (0, 0, 0, 0)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.11224D+01    |proj g|=  1.21697D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      4      6      1     0     0   1.027D-07   9.630D+00
  F =   9.6297422151482568     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
                                     SARIMAX Results                                      
Dep. Variable:                               cost   No. Obs

 This problem is unconstrained.


{'mse': 2065585407924.8767,
 'mae': 1169439.369321766,
 'r2': -1.8107850774768917,
 'RMSE': 1437214.4613539334,
 'MAPE': 0.38663274648689977,
 'SMAPE': 29.85427592797335,
 'MAPE_percent': 38.66}

In [ ]:
#MAL train testpero guia para PIPELINE de 3 modelos
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# # Cargar y preparar los datos
# path_load = "../../data/"
# file_name = "processed/df_time_monthly.csv"
# full_path = path_load + file_name
# data = pd.read_csv(full_path, parse_dates=True)
# data['date'] = pd.to_datetime(data['date'])
# data['date'] = data['date'].dt.tz_localize(None)
# data.set_index('date', inplace=True)
# data.index.freq = 'M'
# # data = pd.read_csv('datos.csv')  # Suponiendo que los datos están en un archivo CSV
# X = data.drop('cost', axis=1)  # Variables exógenas
# y = data['cost']  # Variable objetivo

# # Dividir los datos en conjuntos de entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Definir los modelos a evaluar en el pipeline
# models = [
#     ('Linear Regression', LinearRegression()),
#     ('Random Forest', RandomForestRegressor()),
#     ('Auto ARIMA', pm.auto_arima)
# ]

# # Entrenar y evaluar los modelos en el pipeline
# for model_name, model in models:
#     if model_name == 'Auto ARIMA':
#         model.fit(y_train, exogenous=X_train)
#         y_pred = model.predict(n_periods=len(y_test), exogenous=X_test)
#     else:
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
    
#     mse = mean_squared_error(y_test, y_pred)
#     mae = mean_absolute_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)
    
#     print(f"Model: {model_name}")
#     # Configurar locale para mostrar los valores en pesos colombianos
#     locale.setlocale(locale.LC_MONETARY, 'es_CO.UTF-8')
    
#     print("Mean Squared Error (MSE): {}".format(locale.currency(mse, grouping=True)))
#     print("Mean Absolute Error (MAE): {}".format(locale.currency(mae, grouping=True)))
#     print("R-squared (R2): {:.2f}".format(r2))
#     print('---')



In [ ]:
# Cargar y preparar los datos
data = load_dataframe_from_csv(full_path)
data.index.freq = 'M'
X = data.drop('cost', axis=1)  # Variables exógenas
y = data['cost']  # Variable objetivo
# Dividir los datos en conjuntos de entrenamiento y prueba basado en el índice de tiempo
train_size = int(len(data) * 0.8)
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

# Ajustar modelo Auto ARIMA
model_arima = pm.auto_arima(train_y, exogenous=train_X, seasonal=True)
y_pred_arima = model_arima.predict(n_periods=len(test_y), exogenous=test_X)

# Ajustar modelo SARIMA
order = model_arima.order
seasonal_order = model_arima.seasonal_order
model_sarima = SARIMAX(train_y, exog=train_X, order=order, seasonal_order=seasonal_order)
model_sarima_fit = model_sarima.fit()

# Predecir con SARIMA
y_pred_sarima = model_sarima_fit.predict(start=len(train_y), end=len(train_y) + len(test_y) - 1, exog=test_X)

# Definir los modelos a evaluar en el pipeline
models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest', RandomForestRegressor())
]

# Entrenar y evaluar los modelos en el pipeline
for model_name, model in models:
    model.fit(train_X, train_y)
    y_pred = model.predict(test_X)
    print(f"Model: {model_name}")
    evaluate_forecast(test_y, y_pred)
    print('---')

# Calcular métricas para Auto ARIMA
print("Model: Auto ARIMA")
evaluate_forecast(test_y, y_pred_arima)
print('---')

# Calcular métricas para SARIMA
print("Model: SARIMA")
evaluate_forecast(test_y, y_pred_sarima)